In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# FUNCTION FOR CALCULATING ANGLE
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [3]:
# FRONT RAISE COUNTER
cap = cv2.VideoCapture(0)

# Front counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            
            # Calculate angle
            angle1 = calculate_angle(l_wrist, l_shoulder, l_hip)
            angle2 = calculate_angle(r_wrist, r_shoulder, r_hip)
            
            # Visualize angle
            cv2.putText(image, str(angle1), 
                           tuple(np.multiply(l_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(angle2), 
                           tuple(np.multiply(r_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Front raise counter logic
            if angle1 < 30 and angle2 < 30:
                stage = "Front"
            if angle1 > 80 and stage =='Front' and angle2 > 80:
                stage="Rest"
                counter +=1
                        
            if angle2 < 30 and angle2 < 80:
                stage = "Front"
            if angle2 > 80 and stage =='Front' and angle1 > 80:
                stage="Rest"
                counter +=1
                
            
                       
        except:
            pass
        
              # Render curl counter
        # Setup status box
#         cv2.rectangle(image, (0,0), (225,73), (0,0,0), -1)
#         img = cv2.imread(image)
        x, y, w, h = 10, 10, 120, 80
        sub_img = image[y:y+h, x:x+w]
        white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 255
        res = cv2.addWeighted(sub_img, 0.5, white_rect, 0.6, 1.0)
        image[y:y+h, x:x+w] = res
        font = cv2.FONT_HERSHEY_TRIPLEX

    
        # Rep data
        cv2.putText(sub_img, 'Direction', (12,18), 
                    font, 0.6, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(sub_img, stage, 
                    (12,60), 
                    font, 0.7, (0,0,0), 1, cv2.LINE_AA)
        
        x, y, w, h = 1150, 10, 120, 80
        sub_img = image[y:y+h, x:x+w]
        white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 255
        res = cv2.addWeighted(sub_img, 0.5, white_rect, 0.6, 1.0)
        image[y:y+h, x:x+w] = res
        
        # Stage data
        cv2.putText(sub_img, 'REPS', (30,18), 
                    font, 0.6, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(sub_img, str(counter), 
                    (30,60), 
                    font, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

17.9338512262841
21.130078154252168
24.910314148541772
24.368649616987813
26.093109735362024
26.279027411757674
26.13226849355316
27.745738286848344
26.58212056079338
24.8293610702104
24.96377594121972
24.166540472328865
23.62010084250617
23.86039707453024
24.122674175582322
23.72045096574563
23.76035448883092
23.271271905444294
23.10218247784235
22.706479314179173
22.92949594470313
26.44249611733629
25.12136496515474
23.213997126108175
23.408747029405312
23.37064165445724
23.300327497919206
23.635049298442166
23.76641585927554
24.273488251911957
26.32770457552961
23.685345666883293
23.175293977959456
23.51805499394441
23.24742616749075
60.240488258954436
106.80080876267279
150.18244496077003
155.91665970082266
157.79045775485164
156.26258174170476
156.3760020304686
148.60693245438264
60.17857972121889
32.116802049418304
21.883730887565235
23.579127077211545
23.283807100699928
22.628189676051825
23.212001535234524
24.166255821874273
24.42317172430164
27.845705313167493
77.0521964929344